In [1]:
from DataFrame_maker import crear_dataframe_corrientes_desde_pickle, create_sample_data, leer_datos_de_corrientes_pickle
from prueba import InteractivePlotterTk
import os
# from GraCorrector import GraCorrector

from Gra import GraficoInteractivo
# %matplotlib qt
import pandas as pd
import numpy as np
import importlib
import pickle

import netCDF4
import matplotlib.pyplot as plt
import datetime
import numpy as np
import matplotlib.dates as mdates



In [2]:
%matplotlib tk

In [ ]:
# Rutas 
carpeta = "/Med_2025-2026/Reportes_Edit/Reporte_4.4/2025/12. Diciembre/BMT3-03-T80/DATOS"
archivo = 'msj24_validados_realt_2025120100_2025123123.pkl'
ruta_BMT3_3 = os.path.join(carpeta, archivo)

# carpeta = "/Med_2025-2026/Reportes_Edit/Reporte_2.4/2025/11. Noviembre/BOT2-01-T20/DATOS"
# archivo = 'msj4_validados_realt_2025111919_2025113023.pkl'
# ruta_BOT2_1 = os.path.join(carpeta, archivo)

# carpeta = "/Med_2025-2026/Reportes_Edit/Reporte_2.4/2025/11. Noviembre/BOT2-02-T20/DATOS"
# archivo = 'msj4_validados_realt_2025111821_2025113023.pkl'
# ruta_BOT2_2 = os.path.join(carpeta, archivo)

In [3]:
# Cargar datos
data_BMT3_3 = leer_datos_de_corrientes_pickle(ruta_BMT3_3)
# data_BOT2_1 = leer_datos_de_corrientes_pickle(ruta_BOT2_1)
# data_BOT2_2 = leer_datos_de_corrientes_pickle(ruta_BOT2_2)

[df_BMT3_3,prof] = crear_dataframe_corrientes_desde_pickle(data_BMT3_3)
# [df_BOT2_1,prof] = crear_dataframe_corrientes_desde_pickle(data_BOT2_1)
# [df_BOT2_2,prof] = crear_dataframe_corrientes_desde_pickle(data_BOT2_2)


In [4]:
# Crear DF nuevo con la combinación de los datos y con un tspan fijo
df = pd.DataFrame()
df["tspan"] = pd.date_range(start='2025-12-01 00:00', end='2025-12-31 23:00', freq='h')

df = df.merge(df_BMT3_3[["tspan","Salinidad_MCT"]].rename(columns={"Salinidad_MCT": "Salinidad_BMT3_3"}), on="tspan", how="left")
# df = df.merge(df_BOT2_1[["tspan","Salinidad_MCT"]].rename(columns={"Salinidad_MCT": "Salinidad_BOT2_1"}), on="tspan", how="left")
# df = df.merge(df_BOT2_2[["tspan","Salinidad_MCT"]].rename(columns={"Salinidad_MCT": "Salinidad_BOT2_2"}), on="tspan", how="left")

df.info(show_counts=True)
# print(df.head(3))



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   tspan             744 non-null    datetime64[ns]
 1   Salinidad_BMT3_3  736 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 11.8 KB


In [5]:
app = InteractivePlotterTk(df)
# Iniciar el bucle de la aplicación
app.root.mainloop()
   
# NOTAS PENDIENTES:
# - Arreglar que al eliminar puntos para que no tenga límite el historial de deshacer
# - Arreglar para que solo se grafiquen las columnas elegidas por el usuario
# - Agregar una función que grafique las variables de Rap y Dir --> sería algo como que df.iloc[idx,col][prof]
# - Arreglar el caso de cuando el usuario elimina puntos de Rap o Dir colocar U y V como nans --> df.iloc[idx,col][prof] = np.nan
# - Agregar la funcionalidad de guardar el DataFrame corregido en un archivo .csv o .xlsx, pkl
# - Agregar la funcionalidad de devolver puntos eliminados (no deshacer la última acción, sino devolver puntos específicos)

In [6]:
app.data

,tspan,Salinidad_BMT3_3
0,20423.000000,NaN
1,20423.041667,35.8764
2,20423.083333,35.8264
3,20423.125000,35.7323
4,20423.166667,NaN
...,...,...
739,20453.791667,35.5610
740,20453.833333,35.3837
741,20453.875000,35.6986
742,20453.916667,NaN


In [7]:
# Guardar el DataFrame corregido en un archivo .pkl usando pickle
# Especifica el nombre del archivo
nombre_archivo = 'BMT3_3_corregidos_diciembre2025.pkl'

# Guardar el objeto en el archivo (usando 'with' para asegurar que el archivo se cierre automáticamente)
with open(nombre_archivo, 'wb') as archivo:
    pickle.dump(app.data, archivo)
print(f"¡Datos guardados exitosamente en '{nombre_archivo}'!")

¡Datos guardados exitosamente en 'BMT3_3_corregidos_diciembre2025.pkl'!


In [ ]:
# Cargar el objeto en el archivo (usando 'with' para asegurar que el archivo se cierre automáticamente)
nombre_archivo = 'BMT3_3_corregidos_diciembre2025.pkl'

with open(nombre_archivo, 'rb') as archivo:
    data = pickle.load(archivo)
print(f"¡Datos cargados exitosamente desde '{nombre_archivo}'!")
print(data)

¡Datos cargados exitosamente desde 'mis_datos.pkl'!
            tspan  Salinidad_BMT3_3  Salinidad_BOT2_1  Salinidad_BOT2_2
0    20393.000000               NaN               NaN               NaN
1    20393.041667               NaN               NaN               NaN
2    20393.083333               NaN               NaN               NaN
3    20393.125000               NaN               NaN               NaN
4    20393.166667               NaN               NaN               NaN
..            ...               ...               ...               ...
715  20422.791667           35.2642           36.0300           28.8071
716  20422.833333           36.3749           36.0440           29.1015
717  20422.875000               NaN           34.8633           30.9462
718  20422.916667           35.7395           34.1885           30.4314
719  20422.958333           35.4121           33.7444           29.9702

[720 rows x 4 columns]


In [ ]:
data["tspan"] = mdates.num2date(data["tspan"].to_numpy())
data["tspan"] = pd.to_datetime(data["tspan"])
print(data)

TypeError: unsupported operand type(s) for *: 'Timestamp' and 'float'

In [11]:
ruta = "I:/89_fernando.jose"
file = "BMT2-01-T80-Oceanografico-20081214-20241220.nc"
ruta_completa = os.path.join(ruta, file)

In [12]:
nc = netCDF4.Dataset(ruta_completa)

In [13]:
print(nc)

<class 'netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    dimensions(sizes): dim_lon(1), dim_lat(1), dim_Depth(22), dim_MT(140401)
    variables(dimensions): float64 Latitude(dim_lat), float64 Longitude(dim_lon), float32 Depth(dim_Depth), float64 MT(dim_MT), float64 Date(dim_MT), float32 temperature(dim_MT, dim_Depth), float32 salinity(dim_MT, dim_Depth), float32 u(dim_MT, dim_Depth), float32 v(dim_MT, dim_Depth), float32 current_speed(dim_MT, dim_Depth), float32 current_direction(dim_MT, dim_Depth)
    groups: 


In [14]:
sal = nc["salinity"][:]
tspan = nc["MT"][:]

In [15]:
sal.shape

(140401, 22)

In [16]:
sal[:,2]

masked_array(data=[35.8757, 35.8633, 35.869 , ...,     nan,     nan,
                       nan],
             mask=False,
       fill_value=np.float64(1e+20),
            dtype=float32)

In [17]:
def datenum_to_datetime(datenum):
    """Lee un array en formato datenum de matlab (numeros de orden 7XXXXX) y los cambia al formato datetime de python.
    Por lo general los datos datenum vienen desde un archivo netCDF generado por matlab por lo que pueden venir como un Masked Array.
    Esta Funcion utiliza las librerías datetime y numpy para hacer la conversión de fechas.
    ***********
    by BelloDev
    agregado 2025/04/06
    ultima revision 2025/04/06
    ***********
    """
    # Primero convierte el maskedArray en un array normal de python
    if isinstance(datenum, np.ma.MaskedArray):
        fechas = np.array(datenum.filled(np.nan))
    else:
        fechas = np.array(datenum)

    # Lista para almacenar las fechas convertidas
    fechas_convertidas = []
    
    for idatenum in fechas.flatten():
         # Conversion de idatenum a un valor escalar
        idatenum = float(idatenum)

        # Convierte un número datenum de MATLAB a datetime de Python
        fecha = datetime.datetime.fromordinal(int(idatenum)) + datetime.timedelta(days=idatenum % 1) - datetime.timedelta(days=366)
        #fechas_convertidas.append(fecha)

        # Redondear al minuto más cercano
        fecha = fecha.replace(microsecond=0)
        if fecha.second >= 30:
            fecha += datetime.timedelta(minutes=1)
        
        fecha = fecha.replace(second=0)
        fechas_convertidas.append(fecha)

    # Retorna las fechas convertidas como un array de numpy
    return np.array(fechas_convertidas)

In [18]:
tspan_fix = datenum_to_datetime(tspan)
tspan = pd.to_datetime(tspan_fix)

In [27]:
tspan_num = mdates.date2num(tspan)
plt.plot(tspan_num, sal[:,2])
plt.plot(data["tspan"], data["Salinidad_BMT3_3"])


In [24]:
plt.show()
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))